In [1]:
# Using PuLP dependency to seriously simplify setting up LP problem. It's just a wrapper to help set things up,
#  but it does come with a built-in solver (which we will also use)
from pulp import LpVariable, LpProblem, LpMinimize, lpDot, LpStatus, value
import numpy as np

from src.graph import Graph
from src.graph._preProcessing import connectGraph, removeBackEdges
from src.data.basicTypes import Ingredient, IngredientCollection, Recipe
from factory_graph import ProgramContext

In [2]:
class LPSolver:
    def __init__(self, graph):
        self.graph = graph
        self.variables = []
        # self.variable_idx_counter = 0 # Autogen current "head" index for variable number
        # self.system = []
        self.solved_vars = None # Result from linear solver

        self.lookup = {} # (machine, product, direction, multi_idx) -> variable index
        self.edge_from_perspective_to_index = {} # (edge, machine_id) -> variable index

def graphPreProcessing(self):
    connectGraph(self)
    # if not self.graph_config.get('KEEP_BACK_EDGES', False):
    #     removeBackEdges(self)
    Graph.createAdjacencyList(self)

def linearProgrammingSolver(self: ProgramContext, project_name: str, recipes: list[Recipe], graph_config: dict):
    g = Graph(project_name, recipes, self, graph_config=graph_config)
    self._graph = g # For test access
    graphPreProcessing(g)
    print(g.adj)
    
    


In [3]:
context = ProgramContext()

context.graph_gen = linearProgrammingSolver # Override solver
context.generate_one("power/fish/methane.yaml")

defaultdict(<function createAdjacencyList.<locals>.<lambda> at 0x000001F4E50179C0>, {'source': defaultdict(<class 'list'>, {'O': [('source', '0', 'pams fish')]}), '0': defaultdict(<class 'list'>, {'I': [('source', '0', 'pams fish')], 'O': [('0', '1', 'methane')]}), '1': defaultdict(<class 'list'>, {'I': [('0', '1', 'methane')], 'O': [('1', 'sink', 'biogas')]}), 'sink': defaultdict(<class 'list'>, {'I': [('1', 'sink', 'biogas')]})})


In [4]:
# This is me figuring out edge cases.

"""
- m: centrifuge
  tier: LV
  I:
    pams fish: 1
  O:
    methane: 96
  eut: 5
  dur: 19
  number: 1
  cost_priority:
    pams fish: 1
  # target:
  #   # methane: 250
  #   pams fish: 1
- m: distillery
  tier: LV
  I:
    methane: 125
  O:
    biogas: 375
  eut: 7
  dur: 1
"""

#  

problem = LpProblem("test", LpMinimize)
cost_inputs = ["pams fish"]
recipe_vars = ["pams fish", "methane"]
items = ["pams fish", "methane", "biogas"]
target_vector = [0,0,1000]
recipe_vectors = {
  "fuge": [-1, 96, 0],
  "dist": [0, -125, 375]
}
# How do you figure out which variables should be inputs?
input_vectors = {
  "fish_in": [1, 0, 0],
}
# rec_mat = np.array(list(recipe_vectors.values())+ list(input_vectors.values()))
# display(rec_mat)
# rec_mat.transpose()
all_vectors = recipe_vectors.copy()
all_vectors.update(input_vectors)
# Item constraints are transpose of values of constraint vectors (all vectors)
item_constraints = np.array(list(all_vectors.values())).transpose()


In [5]:
# problem.variables()
# Create variables
recipe_vars = LpVariable.dicts("rec", all_vectors.keys(), lowBound=0, cat="Continuous")

# Add objective
problem += recipe_vars["fish_in"]
# Add constraints
for item_constraint_vec, target in zip(item_constraints, target_vector):
    problem += lpDot(recipe_vars.values(), item_constraint_vec) >= target
# problem += lpDot(x.values(), target_vector) >= 0

problem.solve()
LpStatus[problem.status]

'Optimal'

In [6]:
for v in problem.variables():
    print(v.name, "=", v.varValue)

rec_dist = 2.6666667
rec_fish_in = 3.4722222
rec_fuge = 3.4722222


In [7]:
# That works, now let's mess around with ways to identify input candidates
# Cases that need to be inputs are useful to report the user, 
# but we want to find "eliminated" variables as described by https://github.com/ClaudeMetz/FactoryPlanner/blob/master/modfiles/backend/calculation/matrix_engine.lua
# - that is, variables that are net-zero after all loops/etc and don't need to be system inputs.

# I'm theorizing that the "eliminated" variables might be findable via the reduced row echelon form of the matrix.

from sympy import Matrix
# m = Matrix(np.hstack([item_constraints, np.array(target_vector).reshape(-1,1)]))
# m.rref()

# One example is a loop in the palladium component of the platline,
# which inputs palladium enriched ammonia and loops that and pall met pow dust
# [ammonia, pall enriched ammonia, pall met pow, pall salt, repre pall]
# and cols are:
# [lcr pall met pow recycle, lcr repre, sifter salt recycle]
pall_loop = [
    [-1000, 0,      0],
    [1000,  -9000,  0],
    [-1,    -9,     0.95],
    [0,     16,     -1],
    [0,     2,      0],
]
 
positive_cost_cycle = Matrix(pall_loop)
 
positive_cost_cycle.rref()
#  Oh yeah, duh, rref on an over-defined matrix. Uhhhh
# This matrix converts recipe executions to items produced, 
# and solving it finds necessary recipes executions/unit time to produce desired items
# We're looking for input-item-to-output item conversions. How do we find those?
# - 

(Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0]]),
 (0, 1, 2))

In [8]:
positive_cost_cycle.singular_value_decomposition()

(Matrix([
 [  -0.0123427693881819,     0.999920219926959, -0.00266637715277894],
 [    0.999921784104044,     0.012337578452739, -0.00197762613328934],
 [ 0.000975236276396803,   0.00201218158793192,    0.688745946762978],
 [ -0.00175569605886587,  -0.00179957317684196,   -0.724995172566176],
 [-0.000219462003270191, -0.000224946177417711,   1.0320007302374e-6]]),
 Matrix([
 [9056.0851669197,                0,                0],
 [              0, 993.812582692507,                0],
 [              0,                0, 1.37930382200068]]),
 Matrix([
 [  0.111777170775024,   -0.99373329625207,  3.6777441977233e-6],
 [ -0.993733296258832,  -0.111777170773151, 7.11721275761968e-7],
 [2.96173288126787e-7, 3.73425105498543e-6,   0.999999999992984]]))

In [9]:
positive_cost_cycle * positive_cost_cycle.transpose()

Matrix([
[ 1000000, -1000000,    1000,       0,      0],
[-1000000, 82000000,   80000, -144000, -18000],
[    1000,    80000, 82.9025, -144.95,    -18],
[       0,  -144000, -144.95,     257,     32],
[       0,   -18000,     -18,      32,      4]])

In [10]:
simpler_pos_cost = Matrix([
    [-1, 2],
    [0.25, -1]
])
simpler_pos_cost.rref()
# I don't think this is going to work for determining which should be inputs, but I think I can figure it out using LP

(Matrix([
 [1, 0],
 [0, 1]]),
 (0, 1))

In [11]:
# One example is a loop in the palladium component of the platline,
# which inputs palladium enriched ammonia and loops that and pall met pow dust
# [ammonia, pall enriched ammonia, pall met pow, pall salt, repre pall]
# and cols are:
# [lcr pall met pow recycle, lcr repre, sifter salt recycle]
pall_loop = [
    [-1000, 0,      0],
    [1000,  -9000,  0],
    [-1,    -9,     0.95],
    [0,     16,     -1],
    [0,     2,      0],
]
target_vector = [0, 0, 0, 0, 10]

# New theory: for anything that is ever an input, make an input vector, but tell LP to minimize number used
# Possible upgrade: seek to maximize earliness of inputs in a topological sort or derivative.

In [12]:
# Priorities: [recipe tax, pall enriched ammonia, additional vector cost]
priority_ratio = 9000/0.95 # Smallest/biggest

from pulp import LpProblem, LpMinimize, LpBinary, LpVariable, value, lpSum, lpDot

problem = LpProblem("min input test", LpMinimize)
recipe_vars = LpVariable.dicts("recipe",  ["lcr pall met pow recycle", "lcr repre", "sifter salt recycle"],0)
variables =  ["ammonia", "pall enriched ammonia", "pall met pow", "pall salt", "repre salt"] 
# Filtered only by things that are ever an input
inputs = ["ammonia", "pall enriched ammonia", "pall met pow", "pall salt"] 
explicit_inputs = ["pall enriched ammonia"]
explicit_input_amounts = LpVariable.dicts("input", explicit_inputs, 0)
# Filtered to remove explicit inputs
additional_inputs = ["ammonia", "pall met pow", "pall salt"] 
additional_input_amounts = LpVariable.dicts("input", additional_inputs, 0)
additional_input_switches = LpVariable.dicts("in_switch", additional_inputs, cat=LpBinary)
try:
    for item, recipe_coeffs, target in zip(variables, pall_loop, target_vector):
        # To reach each target amount, use a combination of the recipe outputs and switched inputs
        # The amount for each input is unlimited, but there is a high cost for switching each one on
        if item in explicit_inputs:
            input_term = explicit_input_amounts[item]
        elif item in additional_inputs:
            input_term = additional_input_amounts[item] * additional_input_switches[item]
        else:
            input_term = 0
        problem += lpDot(recipe_vars.values(), recipe_coeffs) + input_term >= target
        
    # Objective, in increasing order of priority:
    # 1: Per-recipe tax
    # (2, n-1): explicit costs to minimize (explicit inputs)
    # n: High-cost additional inputs
    problem += (priority_ratio**0 * lpSum(recipe_vars.values())
        + priority_ratio**1 * explicit_input_amounts["pall enriched ammonia"]
        + priority_ratio**2 * lpSum(additional_input_switches))
except TypeError as e:
    print("Well poop.")
    print(e)

Well poop.
Non-constant expressions cannot be multiplied


c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [13]:
# Little essay about a new approach to solve this which-inputs-should-we-choose problem:

# Alright, so making the linear optimizer solve for minimal inputs while also solving the problem ain't going to work.
# ... because this formulation is quadratic and I can't think of a nice model that isn't.
# ... we could get rid of the toggle and treat new inputs cumulatively as the highest priority class
# ... but that has lots of edge cases and naturally gives a higher weight to minimizing large-number stuff, like fluids.
# ... we could counteract *that* by normalizing quantities (so 9000L of oxygen becomes 9kL of oxygen, or a less nice unit)
# ... but I can't think of a good normalization scheme that doesn't have problems with the different ranges of quantities between recipes.

# New insight: Focus on solving for minimal inputs first, then solve problem.
# Model this as a cover problem: we want to find the smallest set of inputs which reaches the inputs of all recipes.

# We can once again use Linear Optimization to solve this problem.
# We want to find the minimum number of inputs we must provide so that every recipe *can* be run.
# We want to prioritize hitting every recipe, *then* minimizing the number of inputs.
# Note that the linear optimizer might not use all resources if it choses not to include a relevant recipe in a solution.
# Note that if you can run a parent recipe (e.g. ammonia and oxygen for nitric acid), 
# ... you can run descendent recipes (e.g. nitric acid [and ammonia] for ammonium chloride)

# I'm going to model this as a bipartite graph in my head.
# - Every resource that is ever an input to any recipe becomes a requirement (constraint in LP terms)
#       - (one for oxygen, hydrochloric, ilmenite, whatever)
#       - These are nodes on the right-hand side of the graph.
#       - These are the inputs to "cover" with as few new/raw/actual inputs as possible. 
#       - I will call them "requirement" resources to avoid confusing myself.
# - We also take this same list of inputs and make a binary (on or off) variable for every resource
#       - Picture these as nodes on the left-hand side of the graph
#       - Turning on these inputs corresponds to selecting that variable as an input.
#       - I will call them "providable" resources to avoid confusing myself.
# - In graph terms, we put an edge between every providable resource and the requirements it covers. This includes:
#       - The resource itself (providing oxygen obviously covers oxygen requirements)
#       - For every recipe which uses this resource, cover all of its outputs
#       - For all of *those* resources, keep covering - this becomes DFS.
#       - In LP terms, every constraint is sum(every binary variable that covers this required resource) >= 1
# - Finally, the objective is the minimize the sum of all those binary variables (aka find fewest selected inputs)
# - As a preprocessing step, all requirements that are indirectly covered by explicitly provided inputs can be deleted.

# Something in my head tickles that this is NP-Complete, but I don't feel like putting in the work to test/prove it.
# Regardless, LP solvers can totally tackle NPC problems like Knapsack (and they're reasonably optimized about it)

# One big issue with this approach: selecting a valuable resource before a loop 
# (for example, palladium salt dust, which loops around and indirectly supplies *many* recipes)
# ... will have high value and cover many resources. 
# Theory #1: We could use a heuristic-based "cyclic graph toposort" to minimize back-edges (loop-backs)
# ... use that sort to form a DAG that mostly moves ingredient->output, and then use this algorithm
# Theory #2: By the nature of most GTNH recipe graphs, especially when at least one start-of-the-chain ingredient has been provided,
# ... a solution which uses just-before-a-loop resources as an input won't usually be "minimal".
# Further, this algorithm doesn't need to *perfectly* solve this problem, 
# ... because the user can fix wierd behavior by explicitly providing more resources.


In [14]:
# One example is a loop in the palladium component of the platline,
# which inputs palladium enriched ammonia and loops that and pall met pow dust
# [ammonia, pall enriched ammonia, pall met pow, pall salt, repre pall]
# and cols are:
# [lcr pall met pow recycle, lcr repre, sifter salt recycle]
pall_loop = [
    [-1000, 0,      0],
    [1000,  -9000,  0],
    [-1,    -9,     0.95],
    [0,     16,     -1],
    [0,     2,      0],
]
target_vector = [0, 0, 0, 0, 10]
item_recipe_covers = {
    "ammonia": {"lcr pall met pow recycle": {"pall enriched ammonia"}},
    "pall met pow": {"lcr pall met pow recycle": {"pall enriched ammonia"}},
    "pall enriched ammonia": {"lcr repre": {"pall salt", "repre pall"}},
    "pall met pow": {"lcr repre": {"pall salt", "repre pall"}},
    "pall salt":{"sifter salt recycle": {"pall met pow"}}
}
pall_recipes = {
    "lcr pall met pow recycle": {
        "I": {"ammonia":1000, "pall met pow": 1},
        "O": {"pall enriched ammonia": 1000},
    },
    "lcr repre": {
        "I": {"pall enriched ammonia": 9000, "pall met pow": 9},
        "O": {"pall salt": 16, "repre pall": 2},
    },
    "sifter salt recycle": {
        "I": {"pall salt": 1},
        "O": {"pall met pow": 0.95},
    }
}

In [15]:
import re
from collections import Counter
from src.data.loadMachines import recipesFromConfig
def genSlug(self, s, try_acronym=False):
    slug = re.sub(r"[^a-zA-Z0-9_]", "", re.sub(r"\W+", "_", s.lower()))
    words = slug.split("_")
    if len(slug) > 25 or (try_acronym and len(words)>=2):
        return "".join(word[0] for word in words)
    else:
        return slug

def genRecipeNames(recipes):
    node_recipes = {
        node: self.graph.recipes[node]
        for node in self.graph.nodes
        if node not in ["source", "sink"]
    }
    counter = Counter()
    name_map = {"sink": "sink", "source": "source"}
    for node, recipe in node_recipes.items():
        name = genSlug(recipe.machine, True) + "_" + genSlug(recipe.I[0].name)
        if name in counter:
            counter[name] += 1
            name += f"_{counter[name]}"
        name_map[node] = name
    return name_map

In [16]:
def getItemCovers(recipes):
    item_covers = {}

    # Cheeky DFS for each item to figure out what it covers.
    # This could be done all-at-once with some shenanigans, but I think we want to not allow loops on a per-item basis
    # My gut is telling me that will be a slightly more restricted/less-prone-to-error heuristic.
    item_to_covered_recipes = {}
    recipe_to_covered_items = {}
    items_to_cover = set()
    for covered_recipe, io in recipes.items():
        print(covered_recipe, io)
        for ing in io["I"].keys():
            item_to_covered_recipes[ing] = item_to_covered_recipes.get(ing, []) + [covered_recipe]
            items_to_cover.add(ing)
        for out in io["O"].keys():
            recipe_to_covered_items[covered_recipe] = recipe_to_covered_items.get(covered_recipe, []) + [out]

    for item in item_recipe_covers:
        used_recipes = set()
        covered_items = {item}
        frontier = [item]
        while not len(frontier) == 0:
            covered_item = frontier.pop()
            if covered_item not in item_to_covered_recipes: continue # item covers no recipes
            for covered_recipe in item_to_covered_recipes[covered_item]:
                if covered_recipe in used_recipes: continue # Recipe already used (we've looped)
                used_recipes.add(covered_recipe)
                if covered_recipe not in recipe_to_covered_items: continue # recipe covers no items (we deleted its items in preprocessing)
                for outgoing_item in recipe_to_covered_items[covered_recipe]:
                    if outgoing_item in covered_items: continue # We've already hit this item
                    covered_items.add(outgoing_item)
                    frontier.append(outgoing_item)
        item_covers[item] = covered_items
    return item_covers
getItemCovers(pall_recipes)
# Only ammonia covers ammonia (good), but I think this loopy example is a bad example.

lcr pall met pow recycle {'I': {'ammonia': 1000, 'pall met pow': 1}, 'O': {'pall enriched ammonia': 1000}}
lcr repre {'I': {'pall enriched ammonia': 9000, 'pall met pow': 9}, 'O': {'pall salt': 16, 'repre pall': 2}}
sifter salt recycle {'I': {'pall salt': 1}, 'O': {'pall met pow': 0.95}}


{'ammonia': {'ammonia',
  'pall enriched ammonia',
  'pall met pow',
  'pall salt',
  'repre pall'},
 'pall met pow': {'pall enriched ammonia',
  'pall met pow',
  'pall salt',
  'repre pall'},
 'pall enriched ammonia': {'pall enriched ammonia',
  'pall met pow',
  'pall salt',
  'repre pall'},
 'pall salt': {'pall enriched ammonia',
  'pall met pow',
  'pall salt',
  'repre pall'}}

In [22]:
import yaml

bauxite_example_project = "223_bauxite_line.yaml"
recipes = recipesFromConfig(bauxite_example_project)
# getItemCovers(bauxite_recipes)
recipes[0]

[Ingredient(name='bauxite dust', quant=39)]